In [7]:
import polars

polars.enable_string_cache()

In [8]:
from autofeat.dataset import KaggleDataset

dataset = KaggleDataset(
    name="house-prices-advanced-regression-techniques",
)

In [9]:
from autofeat.transform import Aggregate, Cast, Combine, Encode, Identity, Require

inputs = dataset.derive(
    Require(lambda table: table.name != "test.csv")
    .then(Cast())
    .then(Encode())
    .then(Aggregate(by={"Id"}))
    .then(Identity(), Combine()),
)

filters = dataset.table("train.csv").column("Id")

labels = dataset.table("train.csv").column("SalePrice")